# R IRIS Model

 * Wrap an R decision-tree model (using the ```rpart``` library) for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * R

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
!Rscript train.R

Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-r:0.3 r-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/rpart_4.1-15.tar.gz'
Content type 'application/x-gzip' length 639286 bytes (624 KB)
downloaded 624 KB

* installing *source* package ‘rpart’ ...
** package ‘rpart’ successfully unpacked and MD5 sums checked
** libs
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anova.c -o anova.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anovapred.c -o anovapred.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/

** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (rpart)

The downloaded source packages are in
	‘/tmp/RtmpATMOKk/downloaded_packages’
Build completed successfully


In [3]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 r-iris:0.1

89dde681c929faae32ffa2ddd603399ec2ca725134efc9e8395f4cda28ee6698


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[5.926 4.596 3.609 2.013]]
RECEIVED RESPONSE:
data {
  names: "setosa"
  names: "versicolor"
  names: "virginica"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.0
        }
        values {
          number_value: 0.0217
        }
        values {
          number_value: 0.9783
        }
      }
    }
  }
}




In [5]:
!docker rm iris_predictor --force

iris_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Wrap Model and Test

In [13]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-r:0.1 r-iris:0.1

---> Installing application source...
---> Installing dependencies ...
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/rpart_4.1-15.tar.gz'
Content type 'application/x-gzip' length 639286 bytes (624 KB)
downloaded 624 KB

* installing *source* package ‘rpart’ ...
** package ‘rpart’ successfully unpacked and MD5 sums checked
** libs
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anova.c -o anova.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/r-base-udh3fS/r-base-3.5.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c anovapred.c -o anovapred.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG      -fpic  -g -O2 -fdebug-prefix-map=/build/

installing to /usr/local/lib/R/site-library/rpart/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded
* DONE (rpart)

The downloaded source packages are in
	‘/tmp/RtmpYo7Cgt/downloaded_packages’
Build completed successfully


In [14]:
!kubectl create -f r_iris_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [15]:
!kubectl rollout status deploy/r-iris-deployment-r-iris-predictor-0cbaec2

Waiting for deployment "r-iris-deployment-r-iris-predictor-0cbaec2" rollout to finish: 0 of 1 updated replicas are available...
deployment "r-iris-deployment-r-iris-predictor-0cbaec2" successfully rolled out


In [18]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[4.54  2.006 9.755 1.814]]
RECEIVED RESPONSE:
meta {
  puid: "cia8b3kotsetqgpia3h594e6a2"
  requestPath {
    key: "r-iris-classifier"
    value: "r-iris:0.1"
  }
}
data {
  names: "setosa"
  names: "versicolor"
  names: "virginica"
  ndarray {
    values {
      list_value {
        values {
          number_value: 0.0
        }
        values {
          number_value: 0.0217
        }
        values {
          number_value: 0.9783
        }
      }
    }
  }
}




In [15]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
